In [ ]:

# Step 1: Build and Train a Simple Image Classification Model in Python (PyTorch)

# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# from torchvision import transforms
# from torch.utils.data import DataLoader

# # Load a pre-trained model (ResNet18)
# model = torchvision.models.resnet18(pretrained=True)
# model.fc = nn.Linear(model.fc.in_features, 10)  # For 10-class classification

# # Data Transformations
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
# ])

# # Load CIFAR-10 dataset (just an example)
# trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# trainloader = DataLoader(trainset, batch_size=32, shuffle=True)

# # Training loop
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# for epoch in range(5):  # 5 epochs for demonstration
#     model.train()
#     running_loss = 0.0
#     for inputs, labels in trainloader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


1. Exporting a Model to ONNX Format

In [ ]:
# Step 1: Install ONNX and Required Libraries
%pip install onnx onnxruntime -q

Note: you may need to restart the kernel to use updated packages.


In [6]:
#Step 2: Export a PyTorch Model to ONNX
import torch
import torchvision.models as models
import onnx

# Load a pretrained model (e.g., ResNet18)
model = models.resnet18(pretrained=True)
model.eval()  # Set the model to evaluation mode

# Create a dummy input tensor of the correct shape (e.g., for an image of size 224x224 with 3 channels)
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
onnx_path = "resnet18.onnx"
torch.onnx.export(model, dummy_input, onnx_path, verbose=True)

print(f"Model successfully exported to {onnx_path}")


Exported graph: graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %fc.weight : Float(1000, 512, strides=[512, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %onnx::Conv_193 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %onnx::Conv_194 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_196 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_197 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_199 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_200 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_202 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_203 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_205 : Fl

2. Running the ONNX Model Inference with ONNX Runtime

'/Users/kyawkyawoo/Documents/Projects/vehicle inspection system'

In [14]:
#Step 2: Run Inference with ONNX Runtime
import os
import onnxruntime as ort
import numpy as np
import torch
from torchvision import transforms
from PIL import Image

# Load the ONNX model
onnx_model_path = "resnet18.onnx"
ort_session = ort.InferenceSession(onnx_model_path)

# Prepare the input image (assuming you have an image path)

cur_dir = os.getcwd()
parent_dir = os.path.dirname(cur_dir)
image_path = parent_dir + "/data/cat_internet.jpg" 

# Preprocess the image as done during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load and preprocess the image
image = Image.open(image_path)
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Convert the image to a format that ONNX Runtime expects (numpy array)
input_numpy = input_tensor.numpy()

# Run inference
inputs = {ort_session.get_inputs()[0].name: input_numpy}
outputs = ort_session.run(None, inputs)

# Post-processing (e.g., getting the predicted class)
predicted_class = np.argmax(outputs[0])

print(f"Predicted class: {predicted_class}")


Predicted class: 285


3. Deploying the ONNX Model in Production

In [16]:
%pip install flask -q

Note: you may need to restart the kernel to use updated packages.


In [19]:

from flask import Flask, request, jsonify
import onnxruntime as ort
import numpy as np
from PIL import Image
from torchvision import transforms
import io

# Initialize Flask app
app = Flask(__name__)

# Load the ONNX model
onnx_model_path = "resnet18.onnx"
ort_session = ort.InferenceSession(onnx_model_path)

# Preprocess function
def preprocess_image(image_bytes):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = Image.open(io.BytesIO(image_bytes))
    input_tensor = transform(image).unsqueeze(0)
    return input_tensor.numpy()

# Define the prediction route
@app.route("/predict", methods=["POST"])
def predict():
    if 'file' not in request.files:
        return jsonify({"error": "No file part"}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No selected file"}), 400

    # Get the image bytes from the request
    image_bytes = file.read()
    input_numpy = preprocess_image(image_bytes)

    # Run inference with ONNX Runtime
    inputs = {ort_session.get_inputs()[0].name: input_numpy}
    outputs = ort_session.run(None, inputs)

    # Get the predicted class (assuming single output)
    predicted_class = np.argmax(outputs[0])

    return jsonify({"predicted_class": int(predicted_class)})

# Start the Flask app
# if __name__ == "__main__":
#     app.run(debug=True)

if __name__ == "__main__":
    app.run(debug=True, port=5001)  # Use a different port


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/Users/kyawkyawoo/miniconda3/envs/python_310_env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/kyawkyawoo/miniconda3/envs/python_310_env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/kyawkyawoo/miniconda3/envs/python_310_env/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kyawkyawoo/miniconda3/envs/python_310_env/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/kyawkyawoo/miniconda3/envs/python_310_env/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/Users/kyawkyawoo/miniconda3/envs/python_310_env/lib/python3.10/s

SystemExit: 1

In [ ]:

# Step 3: Integrating PyTorch Model with WebAssembly
# 3.1 Convert PyTorch Model to ONNX Format

import torch
import torchvision.models as models

# Load pre-trained model
model = models.resnet18(pretrained=True)

# Set model to evaluation mode
model.eval()

# Create dummy input tensor
dummy_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX
torch.onnx.export(model, dummy_input, "resnet18.onnx", verbose=True)
